<h2> CS 4630 Final project </h2>

In [14]:
#Importing Libs
import requests
import os, sys
import numpy as np
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup

In [27]:
#Start by pulling in our data from https://www.basketball-reference.com/leagues/NBA_2022_totals.html
#start with giving it the URL
projectURL = "https://www.basketball-reference.com/leagues/NBA_2022_totals.html"
projectPage = urllib.request.urlopen(projectURL).read()
projectSoup = BeautifulSoup(projectPage)

#Next give it the table we want (might add more later on and use 2 tables but this is a start)
basketballTable = projectSoup.find(id = "div_totals_stats")
info = basketballTable.find_all(class_ = "full_table")
basketballDataBody = basketballTable.find("tbody")
basketballTables = basketballDataBody.find_all("tr")

#printing
playerData = pd.DataFrame(columns=["Name", "Age", "Position", "Team", "FG%","3PT%","Games", "Games Started", 'Mins played',
                                   "FG", "FGA", "FG3", "FG3A",'FG2','FG2A','FG2%','EFF%','FT','FTA','FT%','ORB','DRB','TotRB',
                                  'AST','STL','BLK','TOV','PF','Pts'])
#playerData['Player Name'] = playerName

#Adding player data into a pandas dataframe
for i in range(len(info)):
    playerDataDict = {
        "Name" : info[i].find("td", {"data-stat" : "player"}).text,
        "Age" : info[i].find("td", {"data-stat" : "age"}).text,
        "Position" : info[i].find("td", {"data-stat" : "pos"}).text,
        "Team" : info[i].find("td", {"data-stat" : "team_id"}).text,
        "FG%" : info[i].find("td", {"data-stat" : "fg_pct"}).text,
        "3PT%" : info[i].find("td", {"data-stat" : "fg3_pct"}).text,
        "Games" : info[i].find("td", {"data-stat" : "g"}).text,
        "Games Started" : info[i].find("td", {"data-stat" : "gs"}).text,
        "Mins played" : info[i].find("td", {"data-stat" : "mp"}).text,
        "FG" : info[i].find("td", {"data-stat" : "fg"}).text,
        "FGA" : info[i].find("td", {"data-stat" : "fga"}).text,
        "FG3" : info[i].find("td", {"data-stat" : "fg3"}).text,
        "FG3A" : info[i].find("td", {"data-stat" : "fg3a"}).text,
        "FG2" : info[i].find("td", {"data-stat" : "fg2"}).text,
        "FG2A" : info[i].find("td", {"data-stat" : "fg2a"}).text,
        "FG2%" : info[i].find("td", {"data-stat" : "fg2_pct"}).text,
        "EFF%" : info[i].find("td", {"data-stat" : "efg_pct"}).text,
        "FT" : info[i].find("td", {"data-stat" : "ft"}).text,
        "FTA" : info[i].find("td", {"data-stat" : "fta"}).text,
        "FT%" : info[i].find("td", {"data-stat" : "ft_pct"}).text,
        "ORB" : info[i].find("td", {"data-stat" : "orb"}).text,
        "DRB" : info[i].find("td", {"data-stat" : "drb"}).text,
        "TotRB" : info[i].find("td", {"data-stat" : "trb"}).text,
        "AST" : info[i].find("td", {"data-stat" : "ast"}).text,
        "STL" : info[i].find("td", {"data-stat" : "stl"}).text,
        "BLK" : info[i].find("td", {"data-stat" : "blk"}).text,
        "TOV" : info[i].find("td", {"data-stat" : "tov"}).text,
        "PF" : info[i].find("td", {"data-stat" : "pf"}).text,
        "Pts" : info[i].find("td", {"data-stat" : "pts"}).text,
        
    }
    playerData.loc[len(playerData)] = playerDataDict
#Printing the whole table in    
playerData



,Name,Age,Position,Team,FG%,3PT%,Games,Games Started,Mins played,FG,...,FT%,ORB,DRB,TotRB,AST,STL,BLK,TOV,PF,Pts
0,Precious Achiuwa,22,C,TOR,.439,.359,73,28,1725,265,...,.595,146,327,473,82,37,41,84,151,664
1,Steven Adams,28,C,MEM,.547,.000,76,75,1999,210,...,.543,349,411,760,256,65,60,115,153,528
2,Bam Adebayo,24,C,MIA,.557,.000,56,56,1825,406,...,.753,137,427,564,190,80,44,148,171,1068
3,Santi Aldama,21,PF,MEM,.402,.125,32,0,360,53,...,.625,33,54,87,21,6,10,16,36,132
4,LaMarcus Aldridge,36,C,BRK,.550,.304,47,12,1050,252,...,.873,73,185,258,42,14,47,44,78,607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,Thaddeus Young,33,PF,TOT,.518,.354,52,1,845,141,...,.469,80,128,208,104,54,18,52,81,322
601,Trae Young,23,PG,ATL,.460,.382,76,76,2652,711,...,.904,50,234,284,737,72,7,303,128,2155
602,Omer Yurtseven,23,C,MIA,.526,.091,56,12,706,130,...,.623,85,209,294,49,17,20,41,84,299
603,Cody Zeller,29,C,POR,.567,.000,27,0,355,51,...,.776,50,75,125,22,8,6,19,56,140


<h2> Evaluation </h2>
<p> The chart above just shows the players basic attributes </p>

In [26]:
topFgPct = playerData

topFgPct.columns
my_cols = ['Name','Position','Team','FG%']

topFgPct = topFgPct[my_cols]

topFgPct = topFgPct[topFgPct['FG%'] > '.700']

topFgPct

,Name,Position,Team,FG%
22,Udoka Azubuike,C,UTA,.755
98,Ahmad Caver,SG,IND,1.000
194,Rudy Gobert,C,UTA,.713
284,Joe Johnson,SG,BOS,1.000
477,Mitchell Robinson,C,NYK,.761
505,Jericho Sims,PF,NYK,.722
514,Jaden Springer,SG,PHI,1.000
522,Craig Sword,SG,WAS,.750
529,Tyrell Terry,PG,MEM,1.000
588,Robert Williams,C,BOS,.736
